In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import yfinance as yf
import json

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_info'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [5]:
unique_tickers = bigquery.run_query(
    f"""
    SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_sp500_tickers
    UNION DISTINCT
    SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_russell_2000_tickers
    UNION DISTINCT
    SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_nasdaq_tickers
    """
)

unique_tickers

,ticker
0,AACG
1,AADI
2,AADR
3,AAGR
4,AAGRW
...,...
5962,TDS
5963,GCI
5964,GTN
5965,DLX


In [6]:
import yfinance as yf
import pandas as pd

data = []

for i, row in unique_tickers.iterrows():
    ticker = row['ticker']
    
    # Obtener datos históricos
    ticker_data = yf.Ticker(ticker).info
    
    # Convertir el diccionario en una fila de DataFrame
    data_row = pd.Series(ticker_data)
    
    # Agregar el símbolo como una columna en los datos históricos
    data_row['ticker'] = ticker
    
    # Añadir los datos a la lista
    data.append(data_row)

# Concatenar todas las filas y convertir en un DataFrame
df = pd.DataFrame(data)
df.head()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZAZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZAZZT&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZBZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZBZZT&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZCZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZCZZT&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZXYZ.A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomai

,address1,address2,city,zip,country,phone,fax,website,industry,industryKey,...,earningsQuarterlyGrowth,pegRatio,earningsGrowth,dividendRate,dividendYield,payoutRatio,trailingAnnualDividendRate,trailingAnnualDividendYield,grossProfits,industrySymbol
0,Building No. 3,"Room 507 BinhuZhuoyueCheng, WenhuaKechuangYuan...",Hefei,230051,China,86 55 1651 35763,86 10 6517 9517,https://ir.atai.net.cn/,Education & Training Services,education-training-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17383 Sunset Boulevard,Suite A250,Pacific Palisades,90272,United States,424 744 8055,NaN,https://aadibio.com,Biotechnology,biotechnology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,(202) 684-6383,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,445 Park Avenue,Ninth Floor,New York,10022,United States,212 745 1164,NaN,https://africanagriculture.com,Farm Products,farm-products,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,445 Park Avenue,Ninth Floor,New York,10022,United States,212 745 1164,NaN,https://africanagriculture.com,Farm Products,farm-products,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Identificar columnas de tipo 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Convertir las columnas de tipo 'object' a listas
for col in object_columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

In [13]:
df

,address1,address2,city,zip,country,phone,fax,website,industry,industryKey,...,earningsQuarterlyGrowth,pegRatio,earningsGrowth,dividendRate,dividendYield,payoutRatio,trailingAnnualDividendRate,trailingAnnualDividendYield,grossProfits,industrySymbol
0,Building No. 3,"Room 507 BinhuZhuoyueCheng, WenhuaKechuangYuan...",Hefei,230051,China,86 55 1651 35763,86 10 6517 9517,https://ir.atai.net.cn/,Education & Training Services,education-training-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17383 Sunset Boulevard,Suite A250,Pacific Palisades,90272,United States,424 744 8055,NaN,https://aadibio.com,Biotechnology,biotechnology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,(202) 684-6383,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,445 Park Avenue,Ninth Floor,New York,10022,United States,212 745 1164,NaN,https://africanagriculture.com,Farm Products,farm-products,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,445 Park Avenue,Ninth Floor,New York,10022,United States,212 745 1164,NaN,https://africanagriculture.com,Farm Products,farm-products,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962,30 North LaSalle Street,Suite 4000,Chicago,60602,United States,312 630 1900,NaN,https://www.tdsinc.com,Telecom Services,NaN,...,NaN,-0.56,NaN,0.16,0.0072,1.7439,0.60,0.027003,NaN,NaN
5963,175 Sully's Trail,Suite 203,Pittsford,14534-4560,United States,585-598-0030,NaN,https://www.gannett.com,Publishing,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5964,"4370 Peachtree Road, NE",Suite 400,Atlanta,30319,United States,404 504 9828,NaN,https://www.gray.tv,Broadcasting,NaN,...,4.500,0.03,NaN,0.32,0.0614,4.5714,0.32,0.061420,NaN,NaN
5965,801 South Marquette Avenue,NaN,Minneapolis,55402-2807,United States,651 483 7111,NaN,https://www.deluxe.com,Advertising Agencies,NaN,...,0.249,0.82,0.223,1.20,0.0588,1.4118,1.20,0.058795,NaN,NaN


In [21]:
# Lista de columnas a eliminar
columns_to_drop = ['priceToSalesTrailing12Months']
# columns_to_drop = ['forwardPE', 'priceToSalesTrailing12Months']


# Eliminar las columnas
df = df.drop(columns=columns_to_drop)

In [25]:
df["trailingPE"] = df["trailingPE"].astype(str)

In [26]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
